[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap05/cap05-main.ipynb)

# Capítulo 5 - Q-Learning, SARSA e Expected-SARSA


## 1. Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg

    !pip install gymnasium, moviepy

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

### Imports Gerais

In [ ]:
import gymnasium as gym
import numpy as np

In [ ]:
from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path
from util.qtable_helper import evaluate_qtable, record_video_qtable

In [ ]:
# define se os algoritmos vão imprimir dados do treinamento
VERBOSE = True

## 1. Q-Learning

Veremos aqui, vários algoritmos da família *Temporal-Difference Learning* de 1 passo. Todos eles se baseiam na em uma estimativa da função $Q(s,a)$. Porém, diferente dos Monte Carlo, eles apresendem a cada passo de interação com o episódio.

Ao invés do retorno parcial real $G_t$, todos usam uma estimativa *bootstraped* baseada nas equações de Bellamn.

Após uma experiência $(s,a,r,s')$, eles atualizam $Q$ assim:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \times (r + \gamma V(s') - Q(s,a))
$$

O método **Q-Learning** tem como característica a suposição de que, a partir de s', será executada a ação "gulosa". Assim, ele assume que um $s'$ não-terminal terá seu valor dado pela ação de melhor valor (retorno futuro a partir dela):

$$
V(s') = max_{a'}(Q(s',a'))
$$

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def epsilon_greedy(Q, state, epsilon):
    num_actions = len(Q[state])
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])   # em caso de empates, retorna sempre o menor índice --> mais eficiente, porém...
        #return np.random.choice(np.where(Q[state] == Q[state].max())[0]) # aleatoriza em caso de empates

In [ ]:
# Algoritmo Q-learning
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_qlearning(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q toda com zeros
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    all_episode_rewards = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, trunc, _ = env.step(action)
            done = terminated or trunc

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state

        all_episode_rewards.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(all_episode_rewards[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return all_episode_rewards, Q

In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 8_000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.05

env = gym.make(ENV_NAME, max_episode_steps=500)

# Roda o algoritmo Q-Learning
rewards1, qtable1 = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON)
clear_output()

print("Últimos resultados: media =", np.mean(rewards1[-20:]), ", desvio padrao =", np.std(rewards1[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards1, r_max)

In [ ]:
# Avalia, usando a tabela de forma greedy
evaluate_qtable(env, qtable1, num_episodes=10, epsilon=0.0, verbose=True);

In [ ]:
# Salva vídeos, usando a tabela de forma greedy
record_video_qtable(ENV_NAME, qtable1, episodes=2, folder='videos/', prefix='qlearning', epsilon=0.0)

In [ ]:
# Exibe o vídeo gravado
display_videos_from_path('videos/', prefix='qlearning')

## 2. SARSA

O **SARSA** é um algoritmo semelhante ao Q-Learning. Porém, ele usa um valor de $V(s')$ baseado na próxima ação ($a'$) realmente escolhida.

Em outras palavras, o Q-Learning:
- usa os dados $(s,a,r,s')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é (guloso) baseado na melhor ação possível: $V(s') = max_{a'}{Q(s', a')}$

Porém, o SARSA:
- usa os dados $(s,a,r,s',a')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é o resultado da verdadeira próxima ação $a'$: $V(s') = Q(s', a')$

Crie o código do SARSA abaixo, com base no código do Q-Learning. Dicas:
- para atualizar o $Q$, você precisa desta 4 variáveis coerentemente mantidas: `state, action, next_state, next_action`
- escolha a "próxima ação" imediatamente após cada passo (e logo após o reset) e antes da atualização do $Q$

In [ ]:
# Algoritmo SARSA
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_sarsa(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    all_episode_rewards = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()
        action = epsilon_greedy(Q, state, epsilon)

        while not done:
            next_state, reward, terminated, trunc, _ = env.step(action)
            done = terminated or trunc
        
            next_action = epsilon_greedy(Q, next_state, epsilon)

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais
                V_next_state = Q[next_state, next_action]

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state
            action = next_action

        all_episode_rewards.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(all_episode_rewards[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return all_episode_rewards, Q

Se não conseguir implementar, descomente o código abaixo para prosseguir:

In [ ]:
#from cap05.qlearning_sarsa import run_sarsa

In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 8_000
LR = 0.05
GAMMA = 0.95
EPSILON = 0.05

env = gym.make(ENV_NAME, max_episode_steps=500)

# Roda o algoritmo SARSA
rewards2, qtable2 = run_sarsa(env, EPISODES, LR, GAMMA, EPSILON)
clear_output()

print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
plot_result(rewards2, r_max)

In [ ]:
evaluate_qtable(env, qtable2, num_episodes=10, epsilon=0.0, verbose=True);

In [ ]:
record_video_qtable(ENV_NAME, qtable2, episodes=2, folder='videos/', prefix='sarsa', epsilon=0.0)
#clear_output()

In [ ]:
display_videos_from_path('videos/', prefix='sarsa')

## 3. Expected-SARSA

O Expected-SARSA tem semelhanças com os dois algoritmos vistos antes:
- usa os dados $(s,a,r,s')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é o valor médio, com base nas probabilidades dadas pela política:
   $$V(s') = \displaystyle\sum_{a'} \pi(a'|s') \times Q(s',a')$$

Para implementá-lo, precisamos das probabilidades da política realizar cada ação.

A função abaixo retorna as probabilidades de cada ação na política *epsilon-greedy*:

In [ ]:
def epsilon_greedy_probs(Q, state, num_actions, epsilon):
    # lista com a probabilidade que todas as ações têm de ser escolhidas nas decisões exploratórias (não-gulosas)
    probs = [ (epsilon / num_actions) ] * num_actions

    # adiciona a probabilidade para a ação 'greedy'
    # em caso de empate, só o menor índice é considerado
    # (isso é coerente com a implementação da função epsilon_greedy())
    greedy_action = np.argmax(Q[state])
    probs[greedy_action] += (1.0 - epsilon)

    return probs

In [ ]:
# Algoritmo Expected-SARSA
def run_expected_sarsa(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1):
    num_actions = env.action_space.n

    # inicializa a tabela Q toda com zeros
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, trunc, _ = env.step(action)
            done = terminated or trunc

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor esperado
                p_next_actions = epsilon_greedy_probs(Q, next_state, num_actions, epsilon)
                V_next_state = np.sum( np.asarray(p_next_actions) * Q[next_state] )

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state

        # salva o retorno do episódio que encerrou
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q


In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 8_000
LR = 0.2
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME, max_episode_steps=500)

rewards3, qtable3 = run_expected_sarsa(env, EPISODES, LR, GAMMA, EPSILON)
clear_output()

print("Últimos resultados: media =", np.mean(rewards3[-20:]), ", desvio padrao =", np.std(rewards3[-20:]))

In [ ]:
plot_result(rewards3, r_max)

In [ ]:
evaluate_qtable(env, qtable3, num_episodes=10, epsilon=0.0, verbose=True);

In [ ]:
record_video_qtable(ENV_NAME, qtable3, episodes=2, folder='videos/', prefix='expec-sarsa', epsilon=0.0)
clear_output()

In [ ]:
display_videos_from_path('videos/', prefix='expec-sarsa')

## 4. Experimentos

In [ ]:
# parâmetros para todos os algoritmos (não é o ideal)
LR = 0.1
GAMMA = 0.95
EPSILON = 0.05

# silencia os algoritmos
VERBOSE = False

# repetições de cada algoritmo
RUNS = 5

# para carregar automaticamente os resultados de simulações já executadas
AUTO_LOAD = False

### Taxi-v3 (discreto)

In [ ]:
environment = gym.make("Taxi-v3")
NUM_EPISODES = 5_000

results_t = []

results_t.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_t.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_t.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_t, cumulative=False, x_log_scale=False, window=50)

### FrozenLake-v1

In [ ]:
environment = gym.make("FrozenLake-v1")
NUM_EPISODES = 8_000

results_f = []

results_f.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_f.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_f.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_f, cumulative=False, x_log_scale=False, window=100)

### Cliff Walking

In [ ]:
environment = gym.make("CliffWalking-v0", max_episode_steps=100)
NUM_EPISODES = 1_000

results_c = []

results_c.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_c.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()
results_c.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_c, cumulative=False, x_log_scale=False, window=100)